In [3]:
from transformers import AutoFeatureExtractor, ASTForAudioClassification
import torch
import pickle

In [5]:
with open('AutoFeatureExtractor.pk', 'rb') as f:
    feature_extractor = pickle.load(f)
with open('model.pk', 'rb') as f:
    model = pickle.load(f)

In [6]:
feature_extractor

ASTFeatureExtractor {
  "do_normalize": true,
  "feature_extractor_type": "ASTFeatureExtractor",
  "feature_size": 1,
  "max_length": 1024,
  "mean": -4.2677393,
  "num_mel_bins": 128,
  "padding_side": "right",
  "padding_value": 0.0,
  "return_attention_mask": false,
  "sampling_rate": 16000,
  "std": 4.5689974
}

In [10]:
# Get data
import zipfile
import scipy.io
import pandas as pd
import numpy as np
import os
import re
import matplotlib.pyplot as plt
import mne
import torch
from torchvision import datasets, transforms
import sys
sys.path.insert(0,'/scratch/mcesped/code/NoiseDetection_iEEG/interictal_classifier/')
import features
from scipy import stats
from scipy import signal

In [12]:
# Load zip files
# zip_map = {
#     'fnusa': '/home/mcesped/scratch/Datasets/Dataset_Fnusa_1024.zip', 
#     'mayo': '/home/mcesped/scratch/Datasets/Dataset_Mayo_1024.zip',
#     'UFlorida': '/home/mcesped/scratch/Datasets/Dataset_UFlorida_1024.zip'
# }
srate=1024
zip_map = {
    'fnusa': f'/home/mcesped/scratch/Datasets/Dataset_Fnusa_{srate}.zip', 
    'mayo': f'/home/mcesped/scratch/Datasets/Dataset_Mayo_{srate}.zip',
    'UFlorida': f'/home/mcesped/scratch/Datasets/Dataset_UFlorida_{srate}.zip'
}
filename_map = {
    'fnusa': 'Dataset_Fnusa/', 
    'mayo': 'Dataset_Mayo/',
    'UFlorida': 'Dataset_UFlorida/'
}

df_mayo = pd.read_csv('/scratch/mcesped/Datasets/segments_mayo.csv', sep=",", index_col="index")
# df_florida = pd.read_csv('/scratch/mcesped/Datasets/segments_uflorida.csv', sep=",", index_col="index")
# df_fnusa = pd.read_csv('/scratch/mcesped/Datasets/segments_fnusa.csv', sep=",", index_col="index")
df_total = df_mayo#pd.concat([df_mayo, df_fnusa])

data = {
    'pw': [],
    'noise': [],
    'pathology': [],
    'physiology': []
}

id_to_name={
    0: 'pw',
    1: 'noise',
    2: 'pathology',
    3: 'physiology'
}

df_got = pd.DataFrame(columns=df_total.columns)

for cat_id in (1,2,3):
    # Get df with category 
    df_tmp = df_total.loc[df_total.category_id == cat_id]
    # Random sample
    df_random = df_tmp.sample(n=2).reset_index(drop=True)
    df_got = pd.concat([df_got, df_random])
    # Get cat name
    cat_name = id_to_name[cat_id]
    for id in df_random.index:
        # Get inst and id 
        inst = df_random.loc[id,'institution']
        seg_id  = df_random.loc[id,'segment_id']
        file_name = os.path.join(filename_map[inst], seg_id+'.npy')
        zip_file = zip_map[inst]
        with zipfile.ZipFile(zip_file, mode="r") as f:
            # Get all files
            files = f.namelist()
            # print(len(files))
            # print(file_name)
            # Open 1 files to check
            with f.open(file_name) as myfile:
                # data = pd.read_csv(myfile, sep=",", index_col="Unnamed: 0")
                # print(data.head())
                data_tmp = np.load(myfile)
                # print(data.shape)
            # Append to data
            data[cat_name].append(data_tmp)

In [15]:
inputs = feature_extractor(data['noise'][0], sampling_rate=1024, return_tensors="pt")
inputs

ValueError: The model corresponding to this feature extractor: ASTFeatureExtractor {
  "do_normalize": true,
  "feature_extractor_type": "ASTFeatureExtractor",
  "feature_size": 1,
  "max_length": 1024,
  "mean": -4.2677393,
  "num_mel_bins": 128,
  "padding_side": "right",
  "padding_value": 0.0,
  "return_attention_mask": false,
  "sampling_rate": 16000,
  "std": 4.5689974
}
 was trained using a sampling rate of 16000. Please make sure that the provided `raw_speech` input was sampled with 16000 and not 1024.

In [14]:
data['noise'][0]

array([0.29242923, 0.44591592, 0.22355713, ..., 0.8106826 , 0.74143929,
       0.80599804])